### 시간 구간별 선수 활동량 지표 집계 및 시각화

In [ ]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from src.plot_utils import draw_pitch

##### (1) 가공 데이터 불러오기

In [ ]:
match_id = 1
file = f'data_metrica/Sample_Game_{match_id}/Sample_Game_{match_id}_IntegratedData_Reshaped.csv'
traces = pd.read_csv(file, header=0, index_col=0)
traces

##### (2) 후반전 데이터 시점 조정

In [ ]:
h1_duration = traces[traces['period'] == 1]['time'].max()
h1_duration

In [ ]:
traces[traces['period'] == 2]

In [ ]:
traces.loc[traces['period'] == 2, 'time'] = (traces.loc[traces['period'] == 2, 'time'] - h1_duration).round(2)

##### (3) 시간 구간별 지표 산출

In [ ]:
player_id = 8
player_trace = traces[traces['player_id'] == player_id]

segment_size = 15
time_bins = np.arange(0, 50, segment_size)
time_bins = np.append(time_bins, [100])
speed_bins = [0, 7, 15, 20, 25, 50]

stats_by_time_list = []
for period in player_trace['period'].unique():
    period_trace = player_trace[player_trace['period'] == period]
    
    time_labels = []
    for i in range(len(time_bins)-2):
        if period == 1:
            segment_label = f'{time_bins[i]:02d}-{time_bins[i+1]:02d}'
        else:
            segment_label = f'{time_bins[i]+45:02d}-{time_bins[i+1]+45:02d}'
        time_labels.append(segment_label)
    extra_time_label = '45+' if period == 1 else '90+'
    time_labels.append(extra_time_label)

    period_trace['time_cat'] = pd.cut(period_trace['time'] / 60, bins=time_bins, right=True, labels=time_labels)

    durations = period_trace[['time_cat', 'x']].dropna().groupby('time_cat').count() * 0.04
    distances = period_trace.groupby('time_cat')['distance'].sum()
    durations.columns = ['duration']
    period_stats = pd.concat([durations, distances], axis=1)
    period_stats['dist_1min'] = period_stats['distance'] / period_stats['duration'] * 60

    speed_cats = pd.cut(period_trace['speed'], bins=speed_bins, right=False, labels=np.arange(1, 6))
    distances_by_speed = period_trace.pivot_table('distance', index='time_cat', columns=speed_cats, aggfunc='sum')
    distances_by_speed.columns = [f'zone{i}_dist' for i in distances_by_speed.columns]
    period_stats = pd.concat([period_stats, distances_by_speed], axis=1)

    period_stats['max_speed'] = period_trace.groupby('time_cat')['speed'].max()
    stats_by_time_list.append(period_stats)

stats_by_time = pd.concat(stats_by_time_list).round(2)
stats_by_time

##### (4) 시간 구간별 속도 구간별 뛴 거리 시각화

In [ ]:
plt.figure(figsize=(12, 6))
plt.rcParams.update({'font.size': 15})

n_zones = len(distances_by_speed.columns)
colors = plt.cm.jet(np.linspace(0.9, 0.1, n_zones))

bottom = 0
for i, zone_dist in enumerate(distances_by_speed.columns[::-1]):
    plt.bar(
        stats_by_time.index, stats_by_time[zone_dist],
        bottom=bottom, width=0.4, color=colors[i], label=f'Zone {5-i}'
    )
    if i < n_zones - 1:
        bottom = bottom + stats_by_time[zone_dist]

plt.grid(axis='y', color='k', linestyle='--')
plt.axvline(3.5, color='k', linestyle='--')

plt.xlabel('time_cat')
plt.ylabel('distance')

handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles[::-1], labels[::-1], bbox_to_anchor=(1, 1))

plt.title('Distance by Speed Zone')
plt.show()

### 히트맵 시각화

##### (1) 위치 히트맵 시각화

In [ ]:
x = player_trace['x']
y = player_trace['y']
dense_heatmap = np.histogram2d(y, x, bins=(34, 52), range=[[0, 68], [0, 104]])[0]

draw_pitch('white', 'black')
img = plt.imshow(dense_heatmap[::-1], extent=[0, 104, 0, 68], vmin=0, vmax=500, cmap='RdYlGn_r', alpha=0.8)
plt.colorbar()

plt.title('Location Heatmap')
plt.show()

##### (2) 18-Zone Map 시각화

In [ ]:
grid_size = (3, 6)
sparse_heatmap, yedges, xedges = np.histogram2d(y, x, bins=grid_size, range=[[0, 68], [0, 104]])
sparse_heatmap /= len(x)

draw_pitch('white', 'black')
img = plt.imshow(sparse_heatmap[::-1], extent=[0, 104, 0, 68], vmin=0, vmax=0.2, cmap='RdYlGn_r', alpha=0.8)
plt.colorbar()

for i in range(grid_size[0]):
    for j in range(grid_size[1]):
        text_x = (xedges[j] + xedges[j+1]) / 2
        text_y = (yedges[i] + yedges[i+1]) / 2
        plt.text(text_x, text_y, f'{sparse_heatmap[i, j]:.3f}', ha='center', va='center')

plt.title('18-Zone Map')
plt.show()

##### (3) 방향 히트맵 시각화

In [ ]:
player_running_trace = player_trace[player_trace['speed'] > 15]
vx = player_running_trace['vx']
vy = player_running_trace['vy']

vlim = 8
dense_heatmap = np.histogram2d(vy, vx, bins=(vlim*5, vlim*5), range=[[-vlim, vlim], [-vlim, vlim]])[0]

plt.figure(figsize=(10, 8))
img = plt.imshow(dense_heatmap[::-1], extent=[-vlim, vlim, -vlim, vlim], vmin=0, vmax=100, cmap='jet')
plt.colorbar()

plt.axvline(0, color='w', linestyle='--')
plt.axhline(0, color='w', linestyle='--')

plt.title('Direction Heatmap')
plt.show()